In [1]:
# 모듈 임포트
import pandas as pd
import numpy as np
import glob
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# 컬럼명 정의
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current', 'label']

# 정상 데이터
normal_dir = '../realtime/normal/'
normal_files = glob.glob(os.path.join(normal_dir, '*.csv'))
normal_dfs = [pd.read_csv(file, names=column_names, header=None) for file in normal_files]
normal_data = pd.concat(normal_dfs, ignore_index=True)

# 아크 데이터
arc_dir = '../realtime/arc/'
arc_files = glob.glob(os.path.join(arc_dir, '*.csv'))
arc_dfs = [pd.read_csv(file, names=column_names, header=None) for file in arc_files]
arc_data = pd.concat(arc_dfs, ignore_index=True)

# 전체 병합 및 셔플
train_df = pd.concat([normal_data, arc_data], ignore_index=True)
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [3]:
# 파생 피처
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

train_df = add_features(train_df).dropna()

In [4]:
# 슬라이딩 윈도우로 시퀀스 생성
def create_sequences(data, labels, seq_len=8):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(labels[i+seq_len - 1])  # 마지막 시점의 label 사용
    return np.array(X), np.array(y)

features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']
X_raw = train_df[features].values
y_raw = train_df['label'].values.astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

seq_len = 8
X_seq, y_seq = create_sequences(X_scaled, y_raw, seq_len=seq_len)

# CNN 입력 형태로 reshape: (samples, timesteps, features)
X_seq = X_seq.reshape((X_seq.shape[0], seq_len, len(features)))

In [5]:
# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq)


In [6]:
# 모델 정의
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=(seq_len, len(features))),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(128, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/opt/homebrew/anaconda3/envs/arcenv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-05 19:36:40.663446: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-05-05 19:36:40.663490: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-05 19:36:40.663498: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-05 19:36:40.663518: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-05 19:36:40.663542: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

In [7]:
# 학습
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop]
)

Epoch 1/50


2025-05-05 19:36:46.235291: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


317/317 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.8772 - loss: 0.3916 - val_accuracy: 0.8854 - val_loss: 0.3451
Epoch 2/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8860 - loss: 0.3571 - val_accuracy: 0.8854 - val_loss: 0.3438
Epoch 3/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8864 - loss: 0.3469 - val_accuracy: 0.8854 - val_loss: 0.3372
Epoch 4/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8811 - loss: 0.3495 - val_accuracy: 0.8850 - val_loss: 0.3295
Epoch 5/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8841 - loss: 0.3403 - val_accuracy: 0.8857 - val_loss: 0.3205
Epoch 6/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8846 - loss: 0.3298 - val_accuracy: 0.8939 - val_loss: 0.3163
Epoch 7/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8861 - loss: 0.3296 - val_accuracy: 0.8996 - val_loss: 0.3094
Epoch 8/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8953 - loss: 0.3083 - val_accuracy: 0.900

In [8]:
# 저장
model.save('./model/cnn1d_model.h5')
joblib.dump(scaler, './model/scaler_cnn.joblib')

['./model/scaler_cnn.joblib']

In [1]:
# %%
# TFLite 경량화 및 저장
import tensorflow as tf

# GPU 관련 충돌 방지 (필수 아님, 하지만 권장)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# 모델 로드
model = tf.keras.models.load_model('./model/cnn1d_model.h5')

# 변환기 설정
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # 기본 최적화 (dynamic range quantization)

# 만약 float16 quantization도 시도하고 싶다면 아래 주석을 해제하세요:
# converter.target_spec.supported_types = [tf.float16]

# 변환 수행
tflite_model = converter.convert()

# 저장
with open('./model/cnn1d_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TFLite 변환 및 경량화 완료!")


2025-05-05 19:58:38.850458: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-05-05 19:58:38.850493: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-05 19:58:38.850501: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-05 19:58:38.850523: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-05 19:58:38.850538: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


INFO:tensorflow:Assets written to: /var/folders/k8/zsc42w0n02g8ywyn_nyjq8140000gn/T/tmpnc2j7cjs/assets


INFO:tensorflow:Assets written to: /var/folders/k8/zsc42w0n02g8ywyn_nyjq8140000gn/T/tmpnc2j7cjs/assets


: 